In [ ]:
# mount the drive onto here
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
# select the source and target language code
source_language = "ky"
target_language = "kk" 

# this is for bilingual
experiment_name = "bilingual_baseline" 

os.environ["src"] = source_language 
os.environ["tgt"] = target_language
os.environ["tag"] = experiment_name

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/experiments/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/experiments/%s-%s-%s" % (source_language, target_language, experiment_name)
os.environ['data_path'] = "/content/drive/My Drive/TIL Corpus/bitext"
os.environ['dev_path'] = "/content/drive/My Drive/TIL Corpus/dev_set"
os.environ['test_bible'] = "/content/drive/My Drive/TIL Corpus/test_set/bible"
#os.environ['test_ted'] = "/content/drive/My Drive/TIL Corpus/test_set/ted_test"
#os.environ['test_wmt'] = "/content/drive/My Drive/TIL Corpus/test_set/ted_wmt"



In [ ]:
# check if the drive link ia good
!echo "$gdrive_path"
!echo "$data_path"

/content/drive/My Drive/experiments/ky-kk-bilingual_baseline
/content/drive/My Drive/TIL Corpus/bitext


In [ ]:
# collect all the bitext data files for a language pair given above
!find "$data_path" -name "$src-$tgt.*" >> data.txt
!find "$data_path" -name "$tgt-$src.*" >> data.txt

# this prints how many files it has discovered
! wc -l data.txt

8 data.txt


In [ ]:
 !cat data.txt

/content/drive/My Drive/TIL Corpus/bitext/ted_talks/ky-kk/ky-kk.kk
/content/drive/My Drive/TIL Corpus/bitext/ted_talks/ky-kk/ky-kk.ky
/content/drive/My Drive/TIL Corpus/bitext/bible/kk-ky/kk-ky.ky
/content/drive/My Drive/TIL Corpus/bitext/bible/kk-ky/kk-ky.kk
/content/drive/My Drive/TIL Corpus/bitext/aligned-udhr/kk-ky/kk-ky.ky
/content/drive/My Drive/TIL Corpus/bitext/aligned-udhr/kk-ky/kk-ky.kk
/content/drive/My Drive/TIL Corpus/bitext/mozilla/kk-ky/kk-ky.ky
/content/drive/My Drive/TIL Corpus/bitext/mozilla/kk-ky/kk-ky.kk


In [ ]:
# read the files and merge them
paths = open('data.txt', 'r').readlines()
source = []
target = []
for i in range(0, len(paths), 2):
  
  if paths[i].strip().endswith(source_language):
    source += open(paths[i].strip(), 'r').readlines()
    target += open(paths[i+1].strip(), 'r').readlines()
  else:
    source += open(paths[i+1].strip(), 'r').readlines()
    target += open(paths[i].strip(), 'r').readlines()

assert len(source) == len(target)

!rm data.txt

In [ ]:
# load the universal test sets from bible
!find "$test_bible" -name "$src-$tgt.*" >> test_files.txt
!find "$test_bible" -name "$tgt-$src.*" >> test_files.txt
! wc -l test_files.txt

paths = open('test_files.txt', 'r').readlines()
test_source = []
test_target = []
for i in range(0, len(paths), 2):
  
  if paths[i].strip().endswith(source_language):
    test_source += open(paths[i].strip(), 'r').readlines()
    test_target += open(paths[i+1].strip(), 'r').readlines()
  else:
    test_source += open(paths[i+1].strip(), 'r').readlines()
    test_target += open(paths[i].strip(), 'r').readlines()

assert len(test_source) == len(test_target)
!rm test_files.txt

2 test_files.txt


In [ ]:
# load the universal test sets from ted
!find "$test_ted" -name "$src-$tgt.*" >> test_files.txt
!find "$test_ted" -name "$tgt-$src.*" >> test_files.txt
! wc -l test_files.txt

paths = open('test_files.txt', 'r').readlines()
test_source2 = []
test_target2 = []
for i in range(0, len(paths), 2):
  
  if paths[i].strip().endswith(source_language):
    test_source2 += open(paths[i].strip(), 'r').readlines()
    test_target2 += open(paths[i+1].strip(), 'r').readlines()
  else:
    test_source2 += open(paths[i+1].strip(), 'r').readlines()
    test_target2 += open(paths[i].strip(), 'r').readlines()

assert len(test_source2) == len(test_target2)
!rm test_files.txt

find: ‘’: No such file or directory
find: ‘’: No such file or directory
0 test_files.txt


In [ ]:
# load the universal dev sets
!find "$dev_path" -name "dev.$src-$tgt.*" >> dev_files.txt
!find "$dev_path" -name "dev.$tgt-$src.*" >> dev_files.txt
! wc -l dev_files.txt

paths = open('dev_files.txt', 'r').readlines()
dev_source = []
dev_target = []
for i in range(0, len(paths), 2):
  
  if paths[i].strip().endswith(source_language):
    dev_source += open(paths[i].strip(), 'r').readlines()
    dev_target += open(paths[i+1].strip(), 'r').readlines()
  else:
    dev_source += open(paths[i+1].strip(), 'r').readlines()
    dev_target += open(paths[i].strip(), 'r').readlines()

assert len(dev_source) == len(dev_target)

!rm dev_files.txt

2 dev_files.txt


In [ ]:
# deduplicate the data
clean_source = []
clean_target = []

merged_test = set([(test_source[i], test_target[i]) for i in range(0, len(test_source))])
merged_test2 = set([(test_source2[i], test_target2[i]) for i in range(0, len(test_source2))])
merged_dev = set([(dev_source[i], dev_target[i]) for i in range(0, len(dev_source))])

for s, t in zip(source, target):
  # IMPORTANT for uzbek only. Uncomment otherwise!
  # _t = t.replace('ѓ', 'gʻ')
  # _s = s.replace('ѓ', 'gʻ')

  # if the sentences is a single letter (plus a newline), then discard it
  if len(s) > 2 and len(t) > 2:
    if (s, t) not in merged_test and (s, t) not in merged_test2 and (s, t) not in merged_dev:
        clean_source.append(s)
        clean_target.append(t)

dev_source = []
dev_target = []

for (s, t) in merged_dev:
  if len(s) > 2 and len(t) > 2:
    if (s, t) not in merged_test and (s, t) not in merged_test2:
        dev_source.append(s)
        dev_target.append(t)


In [ ]:
clean_source += dev_source[:1700]
clean_target += dev_target[:1700]

dev_source = dev_source[1700:]
dev_target = dev_target[1700:]

In [ ]:
# check the new length of the data (before deduplication)
print(f"Train set: {len(clean_source)} sentences")
print(f"Dev set: {len(dev_source)} sentences")
print(f"Test set (bible): {len(test_source)} sentences")
print(f"Test set (ted talks): {len(test_source2)} sentences")

Train set: 6486 sentences
Dev set: 505 sentences
Test set (bible): 1000 sentences
Test set (ted talks): 0 sentences


In [ ]:
# load the data into a pandas dataframe
import pandas as pd
df = pd.DataFrame(zip(clean_source, clean_target), columns=['source_sentence', 'target_sentence'])
df_dev = pd.DataFrame(zip(dev_source, dev_target), columns=['source_sentence', 'target_sentence'])
df_bible = pd.DataFrame(zip(test_source, test_target), columns=['source_sentence', 'target_sentence'])
df_ted = pd.DataFrame(zip(test_source2, test_target2), columns=['source_sentence', 'target_sentence'])


# drop duplicate translations
df = df.drop_duplicates()
print(f"Train set (after dedup): {len(df)} sentences")


# drop conflicting translations
df.drop_duplicates(subset='source_sentence', inplace=True)
df.drop_duplicates(subset='target_sentence', inplace=True)

print(f"Train set (after double dedup): {len(df)} sentences")


df_dev = df_dev.drop_duplicates()
df_bible = df_bible.drop_duplicates()
df_ted = df_ted.drop_duplicates()


# Shuffle the data to remove bias in dev set selection.
df_pp = df.sample(frac=1, random_state=42).reset_index(drop=True)
print(df.head(3))

print(f"Train set (after dedup): {len(df_pp)} sentences")
print(f"Dev set: {len(df_dev)} sentences")
print(f"Test set (bible): {len(df_bible)} sentences")
print(f"Test set (ted talks): {len(df_ted)} sentences")

Train set (after dedup): 6478 sentences
Train set (after double dedup): 6432 sentences
                                     source_sentence                                    target_sentence
0  Чындыгында, азыр аларга убакытымдын көпчүлүгүн...  Әсіресе қазіргі уақытта.Әртүрлі халықтардың ті...
1                       Бул кербездик деп ойлошот.\n                 Себебі оны маңызсыз деп санайды.\n
2  Санта Фе институтунда жана Москвада мыкты линг...  Әсіресе дарынды ресей лингвистері Мәскеудегі С...
Train set (after dedup): 6432 sentences
Dev set: 505 sentences
Test set (bible): 1000 sentences
Test set (ted talks): 0 sentences


In [ ]:
# This section does the split between train/dev/test for the parallel corpora then saves them as separate files
import csv

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in df_pp.iterrows():
    src_file.write(row["source_sentence"])
    trg_file.write(row["target_sentence"])
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in df_dev.iterrows():
    src_file.write(row["source_sentence"])
    trg_file.write(row["target_sentence"])
  
with open("test."+source_language, "w") as src_file, open("test."+target_language, "w") as trg_file:
  for index, row in df_bible.iterrows():
    src_file.write(row["source_sentence"])
    trg_file.write(row["target_sentence"])

with open("test2."+source_language, "w") as src_file, open("test2."+target_language, "w") as trg_file:
  for index, row in df_ted.iterrows():
    src_file.write(row["source_sentence"])
    trg_file.write(row["target_sentence"])

# TODO: Doublecheck the format below. There should be no extra quotation marks or weird characters. It should also not be empty.
! head train.*
! head dev.*
! head test.*
! head test2.*

==> train.kk <==
Бұл қызметке өзіміз қабілеттіміз деп ойлайтындай еш негізіміз жоқ. Біздегі бар қабілеттілік өзімізден емес, Құдайдан.
Жай, бос әңгімелерден аулақ жүр. Бұларды айтатындар Құдайдың адал жолынан одан бетер адасып кетеді.
Керісінше, жас шарапты жаңа торсыққа құю керек. (Сонда екеуі де аман қалады.)
— Мырза, қызметшім үйде сал болып қатты қиналып жатыр, — деген өтініш жасады.
Ол аянда Анани есімді біреудің кіріп келіп, көзін ашу үшін үстіне қолын қойғанын көрді, — деді Иеміз.
Иса қолын тигізіп бата берсін деп, бірнеше адам Оған нәрестелерін алып келді. Бірақ мұны көрген шәкірттер оларды жібермеді.
Сонымен қатар, халық санын төмендету тенденциясы социалдық ықпалға тікелей байланысты.
Бірақ жүзімшілер ұлын көргенде өзара ақылдасып: «Бұл — мұрагер ғой! Кәне, жүріңдер, оны өлтірейік, сонда мұрасы біздікі болады!» — деседі.※
Олардың арам ойларын білген Иса:— Екі жүзділер, Мені азғырып сынамақсыңдар, ә? 19 Маған салыққа төлейтін бір күміс теңгені көрсетіңдерші! — деді. Олар Оған 

In [ ]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! pip install joeynmt ; cd joeynmt;
!pip3 install torch==1.7.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html


Cloning into 'joeynmt'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 2822 (delta 5), reused 2 (delta 0), pack-reused 2804
Receiving objects: 100% (2822/2822), 2.84 MiB | 2.22 MiB/s, done.
Resolving deltas: 100% (1928/1928), done.
     |████████████████████████████████| 81kB 3.9MB/s 
     |████████████████████████████████| 71kB 10.0MB/s 
     |████████████████████████████████| 327kB 27.1MB/s 
     |████████████████████████████████| 20.1MB 81.6MB/s 
     |████████████████████████████████| 7.0MB 78.4MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 645kB 45.8MB/s 
     |████████████████████████████████| 215kB 58.3MB/s 
     |████████████████████████████████| 112kB 61.4MB/s 
     |████████████████████████████████| 747kB 53.3MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
  Created wheel for wrapt: filename=wrapt-1.11.1-cp36

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4MB 23kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.7.1+cu101 which is incompatible.
  Found existing installation: torch 1.7.1
    Uninstalling torch-1.7.1:
      Successfully uninstalled torch-1.7.1


In [ ]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 

! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test2.$src > test2.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test2.$tgt > test2.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p "$data_path"
! cp train.* "$data_path"
! cp test.* "$data_path"
! cp test2.* "$data_path"
! cp dev.* "$data_path"
! cp bpe.codes.4000 "$data_path"
! ls "$data_path"

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp test2.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Test language Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.kk	      test2.bpe.ky  test.bpe.kk  test.ky       train.kk
dev.bpe.kk	dev.ky	      test2.kk	    test.bpe.ky  train.bpe.kk  train.ky
dev.bpe.ky	test2.bpe.kk  test2.ky	    test.kk	 train.bpe.ky
bpe.codes.4000	dev.kk	      test2.bpe.ky  test.bpe.kk  test.ky       train.kk
dev.bpe.kk	dev.ky	      test2.kk	    test.bpe.ky  train.bpe.kk  train.ky
dev.bpe.ky	test2.bpe.kk  test2.ky	    test.kk	 train.bpe.ky
BPE Test language Sentences
Себебі (@@ е@@ жел@@ де@@ ) Ж@@ үн@@ іс пайғамбардың Н@@ ине@@ в@@ и қалас@@ ының тұрғ@@ ындар@@ ына керемет бел@@ гі болып көрін@@ ген@@ ін@@ дей, көктен келген Билеуші де осы ұрпақ@@ қа керемет бел@@ гі болып көрін@@ етін болады.※
Е@@ жел@@ гі О@@ ң@@ түс@@ ті@@ к Ар@@ а@@ би@@ я@@ ның патш@@ айым@@ ы※ қия@@ мет күні осы ұрпақ@@ тың адамдар@@ ымен бірге сот алдында тұрып, олардың қи@@ қар@@ лы@@ ғын әш@@ кереле@@ й@@ ді.※ Себебі патш@@ айым С@@ үле@@ й@@ мен@@ нің дан@@ алық сөздер@@ ін тыңда@@ у үшін жер@@ дің қи@@ ыр шет@@ інен келді. Ал мұ

In [ ]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"  # change this to data/{name}/test2.bpe so that you can test it on Ted Talks
    level: "bpe"
    lowercase: False
    max_sent_length: 128
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "none"            # `tokenize` option in sacrebleu.corpus_bleu() function (options include: "none" (use for already tokenized test data), "13a" (default minimal tokenizer), "intl" which mostly does punctuation and unicode, etc) 

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/best.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.998] 
    scheduling: "plateau"           
    patience: 7                     
    learning_rate_factor: 0.5       
    learning_rate_warmup: 4000     
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003          
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 8
    early_stopping_metric: "ppl"
    epochs: 3000                     
    validation_freq: 500          
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "{gdrive_path}/models/{name}_transformer"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    fp16: True
    max_output_length: 128
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             
        embeddings:
            embedding_dim: 256   
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         
        ff_size: 1024            
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             
        embeddings:
            embedding_dim: 256    
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         
        ff_size: 1024           
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

In [ ]:
# This may take a few minutes to install but it will speed up your training a lot!
!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

fatal: destination path 'apex' already exists and is not an empty directory.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-_9lgyivt
Created temporary directory: /tmp/pip-req-tracker-bhnzuovz
Created requirements tracker '/tmp/pip-req-tracker-bhnzuovz'
Created temporary directory: /tmp/pip-install-s3bscp3v
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-4o4jhj18
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-bhnzuovz'
    Running setup.py (path:/tmp/pip-req-build-4o4jhj18/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.7.1+cu101


    running egg_info
    creating /tmp/pip-req-build-4o4jhj18/pip-egg-info/apex.egg-info

In [ ]:
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml


Streaming output truncated to the last 5000 lines.
2021-02-01 08:47:28,296 - INFO - joeynmt.training - Epoch 288, Step:     4300, Batch Loss:     1.410141, Tokens per Sec:    18502, Lr: 0.000300
2021-02-01 08:47:32,510 - INFO - joeynmt.training - Epoch 288: total training loss 22.55
2021-02-01 08:47:32,511 - INFO - joeynmt.training - EPOCH 289
2021-02-01 08:47:45,017 - INFO - joeynmt.training - Epoch 289: total training loss 22.46
2021-02-01 08:47:45,017 - INFO - joeynmt.training - EPOCH 290
2021-02-01 08:47:57,667 - INFO - joeynmt.training - Epoch 290: total training loss 22.35
2021-02-01 08:47:57,668 - INFO - joeynmt.training - EPOCH 291
2021-02-01 08:48:10,086 - INFO - joeynmt.training - Epoch 291: total training loss 22.40
2021-02-01 08:48:10,086 - INFO - joeynmt.training - EPOCH 292
2021-02-01 08:48:22,577 - INFO - joeynmt.training - Epoch 292: total training loss 22.36
2021-02-01 08:48:22,577 - INFO - joeynmt.training - EPOCH 293
2021-02-01 08:48:34,910 - INFO - joeynmt.training 

In [ ]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

In [ ]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"